In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random

In [36]:
# Cargar el modelo preentrenado
base_model = tf.keras.applications.InceptionV3(include_top=False, weights='imagenet')


In [37]:
# Definir las capas de interés
names = ['mixed3', 'mixed5']
layers = [base_model.get_layer(name).output for name in names]

deepdream_model = tf.keras.Model(inputs=base_model.input, outputs=layers)

In [38]:
# Preprocesamiento de imágenes
Sample_Image = tf.keras.preprocessing.image.load_img('stary_night.jpg', target_size=(225, 375))
Sample_Image = tf.keras.preprocessing.image.img_to_array(Sample_Image)
Sample_Image = tf.keras.applications.inception_v3.preprocess_input(Sample_Image)

In [39]:
@tf.function
def calc_loss(image, model):
    img_batch = tf.expand_dims(image, axis=0)
    layer_activations = model(img_batch)
    losses = [tf.reduce_mean(act) for act in layer_activations]
    return tf.reduce_sum(losses)

@tf.function
def deepdream(model, image, step_size):
    with tf.GradientTape() as tape:
        tape.watch(image)
        loss = calc_loss(image, model)

    gradients = tape.gradient(loss, image)
    gradients /= tf.math.reduce_std(gradients) + 1e-8
    image = image + gradients * step_size
    image = tf.clip_by_value(image, -1, 1)
    return loss, image

In [40]:
# Función para ejecutar DeepDream
def run_deep_dream(model, image, steps=100, step_size=0.01):
    for step in range(steps):
        loss, image = deepdream(model, image, step_size)
        if step % 100 == 0:
            plt.figure(figsize=(12, 12))
            plt.imshow(deprocess(image))
            plt.axis('off')
            plt.show()
            print("Step {}, loss {}".format(step, loss))

    plt.figure(figsize=(12, 12))
    plt.imshow(deprocess(image))
    plt.axis('off')
    plt.show()

    return deprocess(image)

def deprocess(image):
    image = 255 * (image + 1.0) / 2.0
    return tf.cast(image, tf.uint8)


In [41]:
dream_img = run_deep_dream(deepdream_model, Sample_Image, steps=2000, step_size=0.001)


Output hidden; open in https://colab.research.google.com to view.